$y = x^3 + 2x^2y -5xy^2 -3y +2$

Step 1. R1CS real numbers to test

In [31]:
import numpy as np
import random
x = 5#random.randint(1,10)
y = 10#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + 2 * v4 - 5 * x * v3 - 3 * y + 2
print(1,out,x,y,v1,v2,v3,v4)

L = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,-5,0,0,0,0,0]
    ])

R = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ])

O = np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [-2,1,0,3,0,-1,0,-2]
    ])

w = np.array([1,out,x,y,v1,v2,v3,v4])

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

1 97 5 10 25 125 100 1250


Step 2. R1CS over Finite Field

In [36]:
import galois
GF = galois.GF(113)
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,108,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [111,1,0,3,0,GF(112),0,GF(111)] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 12 100 7


Step 3. Interpolate columns. 

In [39]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
# print(L_vector_of_poly)

[Poly(0, GF(113)) Poly(0, GF(113))
 Poly(94x^4 + 58x^3 + 71x^2 + 4x, GF(113))
 Poly(85x^4 + 110x^3 + 97x^2 + 37x + 10, GF(113))
 Poly(94x^4 + 21x^3 + 9x^2 + 112x + 103, GF(113))
 Poly(94x^4 + 96x^3 + 12x^2 + 29x + 108, GF(113)) Poly(0, GF(113))
 Poly(0, GF(113))]
